In [146]:
%load_ext autoreload
%autoreload 2

import tabulate
from omegaconf import OmegaConf
import wandb_util.wandb_util as wbu
from scripts.wandb_experiments.benchmark import BenchmarkConfig

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [161]:
from scripts.wandb_experiments.benchmark import GeometryAndPrompts
from text3d2video.util import concat_lists


mv_cat = GeometryAndPrompts(
    "mv_cat_statue_25:latest",
    "cat_statue_mv:latest",
    "cat_statue_mv:latest",
    ["Metalic Cat Statue"],
    1,
)

ymca = GeometryAndPrompts(
    "ymca_20:latest",
    "human_mv:latest",
    "ymca_src:latest",
    ["Stormtrooper", "Deadpool", "Abraham Lincoln"],
    1,
)

rumba = GeometryAndPrompts(
    "rumba_20:latest",
    "human_mv:latest",
    "rumba_src:latest",
    ["Stormtrooper"],
    1,
)


turn_around = GeometryAndPrompts(
    "catwalk_180_20:latest",
    "human_mv:latest",
    "catwalk_180_src:latest",
    ["Deadpool", "Sormtrooper"],
    1,
)

geometries_and_promtps = [
    mv_cat,
    ymca,
    rumba,
    turn_around,
]

scenes = concat_lists(s.to_scenes() for s in geometries_and_promtps)
print(tabulate.tabulate([s.tabulate_row() for s in scenes], headers="keys"))

animation_tag            texturing_tag         src_tag                 prompt                seed
-----------------------  --------------------  ----------------------  ------------------  ------
mv_cat_statue_25:latest  cat_statue_mv:latest  cat_statue_mv:latest    Metalic Cat Statue       0
ymca_20:latest           human_mv:latest       ymca_src:latest         Stormtrooper             0
ymca_20:latest           human_mv:latest       ymca_src:latest         Deadpool                 0
ymca_20:latest           human_mv:latest       ymca_src:latest         Abraham Lincoln          0
rumba_20:latest          human_mv:latest       rumba_src:latest        Stormtrooper             0
catwalk_180_20:latest    human_mv:latest       catwalk_180_src:latest  Deadpool                 0
catwalk_180_20:latest    human_mv:latest       catwalk_180_src:latest  Sormtrooper              0


In [169]:
from scripts.wandb_experiments.benchmark import Method
from scripts.wandb_runs.run_generative_rendering import (
    RunGenerativeRenderingConfig,
    run_generative_rendering,
)
from text3d2video.pipelines.generative_rendering_pipeline import (
    GenerativeRenderingConfig,
)
from text3d2video.pipelines.pipeline_utils import ModelConfig
from text3d2video.utilities.omegaconf_util import get_import_path


decoder_paths = [
    "mid_block.attentions.0.transformer_blocks.0.attn1",
    "up_blocks.1.attentions.0.transformer_blocks.0.attn1",
    "up_blocks.1.attentions.1.transformer_blocks.0.attn1",
    "up_blocks.1.attentions.2.transformer_blocks.0.attn1",
    "up_blocks.2.attentions.0.transformer_blocks.0.attn1",
    "up_blocks.2.attentions.1.transformer_blocks.0.attn1",
    "up_blocks.2.attentions.2.transformer_blocks.0.attn1",
    "up_blocks.3.attentions.0.transformer_blocks.0.attn1",
    "up_blocks.3.attentions.1.transformer_blocks.0.attn1",
    "up_blocks.3.attentions.2.transformer_blocks.0.attn1",
]
# GR base Config
base_gr = OmegaConf.structured(
    RunGenerativeRenderingConfig(
        "",
        "",
        GenerativeRenderingConfig(decoder_paths, num_keyframes=3),
        ModelConfig(),
    )
)

# ControlNet base Config
base_controlnet = OmegaConf.structured(
    RunGenerativeRenderingConfig(
        "",
        "",
        GenerativeRenderingConfig(
            [], do_pre_attn_injection=False, do_post_attn_injection=False
        ),
        ModelConfig(),
    )
)

methods = [
    Method("GR", get_import_path(run_generative_rendering), base_gr),
    # Method("ControlNet", get_import_path(run_generative_rendering), base_controlnet),
]

start_noises = [0, 0.2, 0.4]
for start_noise in start_noises:
    base_render_gr = OmegaConf.structured(
        RunGenerativeRenderingConfig(
            "",
            "",
            GenerativeRenderingConfig(decoder_paths),
            ModelConfig(),
            start_noise_level=start_noise,
        )
    )

    methods.append(
        Method(
            f"RenderThenGR-{start_noise}",
            get_import_path(run_generative_rendering),
            base_render_gr,
        )
    )

print(tabulate.tabulate([m.tabulate_row() for m in methods], headers="keys"))

name              fun                       config
----------------  ------------------------  -------------------------------------------------------
GR                run_generative_rendering  prompt: ''
                                            animation_tag: ''
                                            generative_rendering:
                                              module_paths:
                                              - mid_block.attentions.0.transformer_blocks.0.attn1
                                              - up_blocks.1.attentions.0.transformer_blocks.0.attn1
                                              - up_blocks.1.attentions.1.transformer_blocks.0.attn1
                                              - up_blocks.1.attentions.2.transformer_blocks.0.attn1
                                              - up_blocks.2.attentions.0.transformer_blocks.0.attn1
                                              - up_blocks.2.attentions.1.transformer_blocks.0.attn1
        

In [171]:
from scripts.wandb_experiments.benchmark import benchmark


config = BenchmarkConfig(scenes, methods)
config = OmegaConf.structured(config)

spec = benchmark(config)

: 

In [103]:
from scripts.wandb_experiments.benchmark import benchmark

wbu.sync_experiment(benchmark, config, "benchmark_small_new")

Experiment: https://wandb.ai/romeu/diffusion-3D-features/groups/benchmark_small_new/workspace

Would execute 10 new runs:
- human_mvlatest_Stormtrooper
- cat_statue_mvlatest_SilverCatStatue
- human_mvlatest_Deadpool
- cat_statue_mvlatest_DeadpoolCatStatue
- GR
- GR
- GR
- GR
- GR
- GR

Would delete 4 outdated runs:
- cat_statue_mvlatest_DeadpoolCatStatue (rc8vghnu)
- cat_statue_mvlatest_SilverCatStatue (ciy4uflz)
- human_mvlatest_Stormtrooper    (7zkokcdt)
- human_mvlatest_Deadpool        (3r2vta29)

Do you want to continue? (y/n)
Deleting 4 runs
Executing 10 runs


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: romeu. Use `wandb login --relogin` to force relogin


wandb:   4 of 4 files downloaded.  


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 15/15 [01:15<00:00,  5.02s/it]


...

Process Process-43:
Traceback (most recent call last):
  File "/home/jorge/miniconda3/envs/thesis4/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/jorge/miniconda3/envs/thesis4/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/jorge/thesis/wandb_util/wandb_util.py", line 390, in wrapper
    result = func(cfg, run_config, *args, **kwargs)
  File "/home/jorge/thesis/scripts/wandb_runs/run_generative_rendering.py", line 61, in run_generative_rendering
    texture = texture.read_anim_seq()
  File "/home/jorge/thesis/text3d2video/artifacts/anim_artifact.py", line 102, in read_anim_seq
    verts_uvs, faces_uvs = self.uv_data()
  File "/home/jorge/thesis/text3d2video/artifacts/anim_artifact.py", line 86, in uv_data
    verts_uvs = torch.load(self._verts_uvs_path(), weights_only=False)
  File "/home/jorge/miniconda3/envs/thesis4/lib/python3.9/site-packages/torch/serialization.p

wandb:   4 of 4 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
Traceback (most recent call last):
  File "/home/jorge/thesis/wandb_util/wandb_util.py", line 390, in wrapper
    result = func(cfg, run_config, *args, **kwargs)
  File "/home/jorge/thesis/scripts/wandb_runs/run_generative_rendering.py", line 61, in run_generative_rendering
    texture = texture.read_anim_seq()
  File "/home/jorge/thesis/text3d2video/artifacts/anim_artifact.py", line 102, in read_anim_seq
    verts_uvs, faces_uvs = self.uv_data()
  File "/home/jorge/thesis/text3d2video/artifacts/anim_artifact.py", line 86, in uv_data
    verts_uvs = torch.load(self._verts_uvs_path(), weights_only=False)
  File "/home/jorge/miniconda3/envs/thesis4/lib/python3.9/site-packages/torch/serialization.py", line 1065, in load
    with _open_file_like(f, 'rb') as opened_file:
  File "/home/jorge/miniconda3/envs/thesis4/lib/python3.9/site-packages/torch/serialization.py", line 468, in _open

Process Process-44:
Traceback (most recent call last):
  File "/home/jorge/miniconda3/envs/thesis4/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/jorge/miniconda3/envs/thesis4/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/jorge/thesis/wandb_util/wandb_util.py", line 390, in wrapper
    result = func(cfg, run_config, *args, **kwargs)
  File "/home/jorge/thesis/scripts/wandb_runs/run_generative_rendering.py", line 61, in run_generative_rendering
    texture = texture.read_anim_seq()
  File "/home/jorge/thesis/text3d2video/artifacts/anim_artifact.py", line 102, in read_anim_seq
    verts_uvs, faces_uvs = self.uv_data()
  File "/home/jorge/thesis/text3d2video/artifacts/anim_artifact.py", line 86, in uv_data
    verts_uvs = torch.load(self._verts_uvs_path(), weights_only=False)
  File "/home/jorge/miniconda3/envs/thesis4/lib/python3.9/site-packages/torch/serialization.p

wandb:   4 of 4 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
Traceback (most recent call last):
  File "/home/jorge/thesis/wandb_util/wandb_util.py", line 390, in wrapper
    result = func(cfg, run_config, *args, **kwargs)
  File "/home/jorge/thesis/scripts/wandb_runs/run_generative_rendering.py", line 61, in run_generative_rendering
    texture = texture.read_anim_seq()
  File "/home/jorge/thesis/text3d2video/artifacts/anim_artifact.py", line 102, in read_anim_seq
    verts_uvs, faces_uvs = self.uv_data()
  File "/home/jorge/thesis/text3d2video/artifacts/anim_artifact.py", line 86, in uv_data
    verts_uvs = torch.load(self._verts_uvs_path(), weights_only=False)
  File "/home/jorge/miniconda3/envs/thesis4/lib/python3.9/site-packages/torch/serialization.py", line 1065, in load
    with _open_file_like(f, 'rb') as opened_file:
  File "/home/jorge/miniconda3/envs/thesis4/lib/python3.9/site-packages/torch/serialization.py", line 468, in _open

Process Process-45:
Traceback (most recent call last):
  File "/home/jorge/miniconda3/envs/thesis4/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/jorge/miniconda3/envs/thesis4/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/jorge/thesis/wandb_util/wandb_util.py", line 390, in wrapper
    result = func(cfg, run_config, *args, **kwargs)
  File "/home/jorge/thesis/scripts/wandb_runs/run_generative_rendering.py", line 61, in run_generative_rendering
    texture = texture.read_anim_seq()
  File "/home/jorge/thesis/text3d2video/artifacts/anim_artifact.py", line 102, in read_anim_seq
    verts_uvs, faces_uvs = self.uv_data()
  File "/home/jorge/thesis/text3d2video/artifacts/anim_artifact.py", line 86, in uv_data
    verts_uvs = torch.load(self._verts_uvs_path(), weights_only=False)
  File "/home/jorge/miniconda3/envs/thesis4/lib/python3.9/site-packages/torch/serialization.p

In [14]:
from scripts.wandb_experiments.benchmark import split_runs

exp_name = "benchmark_small"

runs = wbu.get_logged_runs(exp_name)
print(wbu.get_exp_url(exp_name))
texture_runs, video_gen_runs = split_runs(runs)

https://wandb.ai/romeu/diffusion-3D-features/groups/benchmark_small/workspace


In [15]:
from text3d2video.util import group_into_array


def scene_key(run):
    config = OmegaConf.create(run.config)
    return f"{config.animation_tag}-{config.prompt}"


def method_key(run):
    return run.name.split("_")[0]


runs_grouped, labels = group_into_array(video_gen_runs, [method_key, scene_key])

# methods = ["ControlNet", "GR"]
# method_indices = [labels[0].index(m) for m in methods]

runs_grouped.shape

(9, 6)

In [16]:
from text3d2video.experiment_analysis import GrData
from text3d2video.util import map_array


run_data = map_array(runs_grouped, GrData.from_gr_run, pbar=True)

  0%|          | 0/54 [00:00<?, ?it/s]

 22%|██▏       | 12/54 [01:00<04:02,  5.78s/it]wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb:   10 of 10 files downloaded.  
wandb:   10 of 10 files downloaded.  4mwandb: \ 1 of 10 files downloaded...
wandb:   20 of 20 files downloaded.  4mwandb: \ 1 of 20 files downloaded...
wandb:   20 of 20 files downloaded.  4mwandb: \ 1 of 20 files downloaded...
wandb:   20 of 20 files downloaded.  4mwandb: \ 1 of 20 files downloaded...
wandb:   20 of 20 files downloaded.  4mwandb: \ 1 of 20 files downloaded...
wandb:   10 of 10 files downloaded.  4mwandb: \ 1 of 10 files downloaded...
wandb:   10 of 10 files downloaded.  4mwandb: \ 1 of 10 files downloaded...
wandb:   20 of 20 files downloaded.  4mwandb: \ 1 of 20 files downloaded...
wandb:   20 of 20 files downloaded.  4mwandb: \ 1 of 20 files downloaded...
wandb:   20 of 20 files downloaded.  4mwandb: \ 1 of 20 files downloaded...
wandb:   20 of 20 files downloaded.  4mwandb:

In [17]:
from scripts.wandb_experiments.benchmark import GrRunData
from text3d2video.clip_metrics import CLIPMetrics
from text3d2video.util import map_array
from functools import partial

clip = CLIPMetrics()

map_array(run_data, partial(GrRunData.compute_clip_metrics, model=clip), pbar=True)
map_array(run_data, GrRunData.compute_uv_mse, pbar=True)

100%|██████████| 54/54 [00:15<00:00,  3.43it/s]


array([[None, None, None, None, None, None],
       [None, None, None, None, None, None],
       [None, None, None, None, None, None],
       [None, None, None, None, None, None],
       [None, None, None, None, None, None],
       [None, None, None, None, None, None],
       [None, None, None, None, None, None],
       [None, None, None, None, None, None],
       [None, None, None, None, None, None]], dtype=object)

In [18]:
from text3d2video.utilities.video_comparison import VideoLabel, add_label_to_clip


def video_with_metrics(d: GrRunData):
    clip = pil_frames_to_clip(d.frames)

    rows = []
    if d.frame_consistency is not None:
        rows.append(f"FC: {d.frame_consistency:.4f}")
    if d.prompt_fidelity is not None:
        rows.append(f"PF: {d.prompt_fidelity:.4f}")
    if d.uv_mse is not None:
        rows.append(f"UV MSE: {d.uv_mse:.4f}")

    content = "\n".join(rows).format(d=d)
    label = VideoLabel(content, font_size=30)
    return add_label_to_clip(clip, label, position=("left", "bottom"))


cropped_run_data = run_data[:, :]
vids = map_array(cropped_run_data, video_with_metrics, pbar=True)

vid = video_grid(vids, y_labels=labels[0])
vid.write_videofile("outs/benchmark_small.mp4")

  0%|          | 0/54 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  2%|▏         | 1/54 [00:00<00:09,  5.34it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  4%|▎         | 2/54 [00:00<00:09,  5.28it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set

Moviepy - Building video outs/benchmark_small.mp4.
Moviepy - Writing video outs/benchmark_small.mp4



Moviepy - Done !
Moviepy - video ready outs/benchmark_small.mp4


In [56]:
import tabulate


table_rows = []

values = []

for row_i, row in enumerate(cropped_run_data):
    method = labels[0][row_i]

    uv_mse = map_array(row, lambda d: d.uv_mse).mean()
    fc = map_array(row, lambda d: d.frame_consistency).mean()
    pf = map_array(row, lambda d: d.prompt_fidelity).mean()

    table_rows.append([method, uv_mse, fc, pf])

    values.append([uv_mse, fc, pf])



headers = ["Method", "UV MSE", "Frame Consistency", "Prompt Fidelity"]
print(tabulate.tabulate(table_rows, headers=headers, tablefmt="markdown"))

Method                 UV MSE    Frame Consistency    Prompt Fidelity
-----------------  ----------  -------------------  -----------------
ControlNet         0.127024               0.899591           0.310699
GR                 0.0202782              0.959487           0.309206
RenderThenCN-0.25  0.0832425              0.92785            0.315243
RenderThenCN-0.5   0.0360604              0.929483           0.310039
RenderThenGR-0     0.0147605              0.953705           0.310851
RenderThenGR-0.25  0.0138913              0.959041           0.301336
RenderThenGR-0.5   0.00714404             0.955827           0.296033
RenderThenGR-0.75  0.00489579             0.956097           0.294004
RenderThenGR-1     0.00148459             0.958688           0.290508
